In [16]:
import pandas as pd
import numpy as np


In [3]:
# import data
prices = pd.read_csv('prices.csv')
sales = pd.read_csv('sales.csv')

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   product_id        175 non-null    int64 
 1   ordered_at        175 non-null    object
 2   quantity_ordered  175 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ KB


In [13]:
#convert objetc to datetime
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])
prices['updated_at'] = pd.to_datetime(prices['updated_at'])

In [6]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   product_id        175 non-null    int64         
 1   ordered_at        175 non-null    datetime64[ns]
 2   quantity_ordered  175 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.2 KB


In [7]:
sales.head()

,product_id,ordered_at,quantity_ordered
0,3998909,2018-09-18 17:51:00,1
1,3998909,2018-09-18 12:52:00,1
2,3998909,2018-09-18 11:33:00,1
3,3998909,2018-09-18 18:47:00,1
4,3998909,2018-09-18 17:36:00,1


In [8]:
prices.head()

,product_id,old_price,new_price,updated_at
0,64,270000,239000,9/10/18 16:37
1,3954203,60000,64000,9/11/18 11:54
2,3954203,60500,57500,9/17/18 22:59
3,3954203,64000,60500,9/15/18 3:49
4,3954203,68800,60000,9/10/18 16:32


In [11]:
sales[sales.product_id==3954203]

,product_id,ordered_at,quantity_ordered
13,3954203,2018-09-11 13:32:00,1
14,3954203,2018-09-11 17:00:00,1
15,3954203,2018-09-11 18:26:00,1
16,3954203,2018-09-11 14:13:00,1
17,3954203,2018-09-11 15:10:00,1
18,3954203,2018-09-12 19:16:00,1
19,3954203,2018-09-11 14:58:00,1
20,3954203,2018-09-11 12:26:00,1
21,3954203,2018-09-11 16:00:00,1
22,3954203,2018-09-11 15:27:00,1


In [14]:
prices[prices.product_id==3954203]

,product_id,old_price,new_price,updated_at
1,3954203,60000,64000,2018-09-11 11:54:00
2,3954203,60500,57500,2018-09-17 22:59:00
3,3954203,64000,60500,2018-09-15 03:49:00
4,3954203,68800,60000,2018-09-10 16:32:00


In [22]:
#def function that return closest price base on product id and order_date
def closest_update_date (id, order_date):
    new_date = prices['updated_at'][prices['product_id'] == id]
    if order_date < new_date.min():
        closest_date = new_date.min()
    else:
        closest_date = new_date[order_date > new_date].max()
    return closest_date

In [24]:
#def function that return closest price base on product id and order_date
def return_closest_price(id,order_date):
    day = closest_update_date(id,order_date)
    if order_date < day:
        closest_price = prices['old_price'][(prices['updated_at']==day) & (prices['product_id']==id)].values[0]
    else: 
        closest_price = prices['new_price'][(prices['updated_at']==day) & (prices['product_id'] == id)].values[0]
    return closest_price

                                                                                                               

In [25]:
#find price corresponding each row and append to update_price array
update_price =[]
for i in np.arange(len(sales)):
    updated_price = return_closest_price(sales.iloc[i]['product_id'], sales.iloc[i]['ordered_at'])
    update_price.append(updated_price)
    

In [26]:
#create and calculate price and revenue column 
sales['price'] = update_price
sales['revenue'] = sales['price']*sales['quantity_ordered']

In [27]:
#export to csv file
print(sales)
sales.to_csv("revenue.csv")

     product_id          ordered_at  quantity_ordered   price  revenue
0       3998909 2018-09-18 17:51:00                 1   16500    16500
1       3998909 2018-09-18 12:52:00                 1   16500    16500
2       3998909 2018-09-18 11:33:00                 1   16500    16500
3       3998909 2018-09-18 18:47:00                 1   16500    16500
4       3998909 2018-09-18 17:36:00                 1   16500    16500
..          ...                 ...               ...     ...      ...
170     4085861 2018-09-13 23:29:00                 1   62500    62500
171          64 2018-09-11 19:32:00                 1  239000   239000
172          64 2018-09-14 20:10:00                 1  239000   239000
173          64 2018-09-12 16:00:00                 1  239000   239000
174          64 2018-09-12 13:23:00                 1  239000   239000

[175 rows x 5 columns]
